# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

In [1]:
! pip install -U langchain

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ---------------------------------------- 0.0/990.0 kB ? eta -:--:--
     -- ------------------------------------- 61.4/990.0 kB ? eta -:--:--
     -- ------------------------------------- 61.4/990.0 kB ? eta -:--:--
     ----------- -------------------------- 286.7/990.0 kB 3.0 MB/s eta 0:00:01
     ------------- ------------------------ 358.4/990.0 kB 2.5 MB/s eta 0:00:01
     ---------------- --------------------- 419.8/990.0 kB 2.0 MB/s eta 0:00:01
     --------------------- ---------------- 553.0/990.0 kB 2.0 MB/s eta 0:00:01
     ------------------------ ------------- 634.9/990.0 kB 2.0 MB/s eta 0:00:01
     ------------------------ ------------- 634.9/990.0 kB 2.0 MB/s eta 0:00:01
     ------------------------ ------------- 634.9/990.0 kB 2.0 MB/s eta 0:00:01
     -------------------------------------- 990.0/990.0 kB 2.2 MB/s eta 0:00:00
     ---------------------------------------- 0.0/373.5 kB ? eta -:--:--
     

## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](../images/llm_chain.png)

## Router Chain: 实现条件判断的大模型调用


这段代码构建了一个可定制的链路系统，用户可以提供不同的输入提示，并根据这些提示获取适当的响应。

主要逻辑：从`prompt_infos`创建多个`LLMChain`对象，并将它们保存在一个字典中，然后创建一个默认的`ConversationChain`，最后创建一个带有路由功能的`MultiPromptChain`。

![](../images/router_chain.png)

In [7]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [39]:
physics_template = """你是一位非常聪明的物理教授。
你擅长以简洁易懂的方式回答关于物理的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""


math_template = """你是一位很棒的数学家。你擅长回答数学问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""

biology_template = """你是一位经验丰富的生物学教授。
你具备深厚的生物学知识，特别在细胞生物学、遗传学和生态学等领域。
你善于将复杂的生物学原理转化成学生们易于理解的概念。
在面对未知的问题时，你会以科学的好奇心去寻找答案。

这是一个问题：
{input}"""

computer_science_template = """你是一名资深的计算机科学教授。
你对算法、数据结构、软件开发和人工智能等领域有着深入的研究。
你能够简洁明了地讲解复杂的计算机概念，并帮助解决技术上的难题。
遇到未知的挑战时，你会借助逻辑和创新性思维进行探索。

这是一个问题：
{input}"""

chinese_literature_template = """你是一位才华横溢的汉语言文学教授。
你精通诗词歌赋、古典文学和现当代文学。
你能够将文学作品中深奥的美学和哲理讲解得生动而富有魅力。
面对学术上的困惑，你会通过扎实的文学理论和历史知识去寻求答案。

这是一个问题：
{input}"""

In [41]:
prompt_infos = [
    {
        "name": "物理",
        "description": "适用于回答物理问题",
        "prompt_template": physics_template
    },
    {
        "name": "数学",
        "description": "适用于回答数学问题",
        "prompt_template": math_template
    },
     { "name": "生物学", 
      "description": "适用于讨论生物学问题", 
      "prompt_template": biology_template,
     }, 
    { "name": "计算机科学", 
     "description": "适用于讨论计算机科学问题",
     "prompt_template": computer_science_template
    }, 
    { "name": "汉语言文学", 
     "description": "适用于分析和讨论汉语言文学相关问题", 
     "prompt_template": chinese_literature_template
    }
]

In [40]:
llm = OpenAI(model_name="gpt-3.5-turbo-instruct",base_url="https://api.xiaoai.plus/v1",api_key="sk-dDREyvruBpYFubUW206d43175fC14aAfBd26A65aAc63E610")

In [42]:
# 创建一个空的目标链字典，用于存放根据prompt_infos生成的LLMChain。
destination_chains = {}

# 遍历prompt_infos列表，为每个信息创建一个LLMChain。
for p_info in prompt_infos:
    name = p_info["name"]  # 提取名称
    prompt_template = p_info["prompt_template"]  # 提取模板
    # 创建PromptTemplate对象
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    # 使用上述模板和llm对象创建LLMChain对象
    chain = LLMChain(llm=llm, prompt=prompt)
    # 将新创建的chain对象添加到destination_chains字典中
    destination_chains[name] = chain

# 创建一个默认的ConversationChain
default_chain = ConversationChain(llm=llm, output_key="text")

In [43]:
type(default_chain)

langchain.chains.conversation.base.ConversationChain

### 使用 LLMRouterChain 实现条件判断调用

这段代码定义了一个chain对象（LLMRouterChain），该对象首先使用router_chain来决定哪个destination_chain应该被执行，如果没有合适的目标链，则默认使用default_chain。

In [26]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [44]:
# 从prompt_infos中提取目标信息并将其转化为字符串列表
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
# 使用join方法将列表转化为字符串，每个元素之间用换行符分隔
destinations_str = "\n".join(destinations)
# 根据MULTI_PROMPT_ROUTER_TEMPLATE格式化字符串和destinations_str创建路由模板
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
# 创建路由的PromptTemplate
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
# 使用上述路由模板和llm对象创建LLMRouterChain对象
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [45]:
print(destinations)

['物理: 适用于回答物理问题', '数学: 适用于回答数学问题', '生物学: 适用于讨论生物学问题', '计算机科学: 适用于讨论计算机科学问题', '汉语言文学: 适用于分析和讨论汉语言文学相关问题']


In [46]:
print(destinations_str)

物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物学: 适用于讨论生物学问题
计算机科学: 适用于讨论计算机科学问题
汉语言文学: 适用于分析和讨论汉语言文学相关问题


In [47]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [48]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物学: 适用于讨论生物学问题
计算机科学: 适用于讨论计算机科学问题
汉语言文学: 适用于分析和讨论汉语言文学相关问题

<< INPUT >>

In [49]:
# 创建MultiPromptChain对象，其中包含了路由链，目标链和默认链。
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [50]:
print(chain.invoke("Redis集群方案什么情况下会导致整个集群不可用?"))



> Entering new MultiPromptChain chain...
计算机科学: {'input': 'Redis集群方案'}
> Finished chain.
{'input': 'Redis集群方案', 'text': '中，什么是数据分片？\n\n数据分片是一种将大量数据分割为较小的数据块，并将每个数据块分配到不同的节点上存储的技术。在Redis集群中，数据分片可以帮助提高系统的性能和扩展性。具体来说，当一个Redis集群中存储的数据量过大时，数据分片可以将数据分散到多个节点上，从而减轻每个节点的负载压力，提高读写效率。同时，数据分片也可以保证数据的高可用性，当某个节点出现故障时，其他节点仍然可以继续提供服务。数据分片的实现可以通过hash算法、一致性哈希算法等方式来确定数据存储在哪个节点上。'}


In [51]:
print(
    chain.invoke(
        "致瘾的生物学基础是什么? "
    )
)



> Entering new MultiPromptChain chain...
生物学: {'input': '致瘾的生物学基础是什么?'}
> Finished chain.
{'input': '致瘾的生物学基础是什么?', 'text': '\n\n致瘾的生物学基础涉及多种因素，包括神经生物学、遗传学、环境因素和心理因素。具体来说，致瘾的生物学基础可以解释为大脑中神经递质的改变，这些改变会导致对某种物质或活动的强烈渴求和依赖。这种渴求和依赖可能是由于遗传因素造成的大脑结构和功能的改变，也可能是由于环境刺激引起的神经适应性改变。此外，心理因素如压力、情绪和个人经历也可能影响一个人对致瘾物质的反应。总的来说，致瘾的生物学基础是一个复杂的综合体，需要多方面的研究来'}


In [53]:
router_chain.verbose = True

In [55]:
print(chain.invoke("我们如何能够在分子水平上理解神经适应性与成瘾之间的关系？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
生物学: {'input': '我们如何能够在分子水平上理解神经适应性与成瘾之间的关系？'}
> Finished chain.
{'input': '我们如何能够在分子水平上理解神经适应性与成瘾之间的关系？', 'text': '\n\n这是一个非常有意思的问题。神经适应性和成瘾之间的关系是一个复杂的问题，需要从多个角度来探讨。首先，我们需要了解神经适应性的定义。神经适应性是指神经系统对环境变化的调整能力，它可以使我们适应不同的环境和压力。成瘾是一种行为模式，它与神经适应性有着密切的联系。\n\n在分子水平上，神经适应性和成瘾之间的关系与神经适应性的改变有关。例如，成瘾物质（如毒品或酒精）可以改变神经适应性，从而导致成瘾行为。这些物质可以影响神经适应性相关的分子，如神经递质和受体'}


In [54]:
print(chain.invoke("汉语与西方语言在形态方面存在哪些显著差异？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
汉语言文学: {'input': '汉语与西方语言在形态方面存在哪些显著差异？'}
> Finished chain.
{'input': '汉语与西方语言在形态方面存在哪些显著差异？', 'text': ' \n\n汉语与西方语言在形态方面的差异主要体现在以下几个方面：\n\n1. 词汇结构：汉语是一种孤立语言，词汇由单个的词语组成，没有词缀和词根的变化，而西方语言则通常具有屈折变化，即通过词缀和词根的变化来表达不同的语法功能。\n\n2. 语序：汉语是一种主谓宾的语序，即主语在前，谓语动词在中间，宾语在后，而西方语言则有更多的语序变化，如英语中的主谓宾、主宾谓、宾主谓等。\n\n3. 语法功能词：汉语中的功能词（如助词、介词、连词等）较少，主要通过语序来表达句子的结构和意'}


### Homework

#### 扩展 Demo：实现生物、计算机和汉语言文学老师 PromptTemplates 及对应 Chains